## Term 3

## Trajectory Generation

In this lesson, We're going to learn about continuous trajectory planning. And more specifically, how to generate drivable trajectories. Quick overview of the lesson:

* define what the motion planning problem is and discuss a few important concepts and plocicies regarding motion planning algorithms. 
* a quick review of A* to get ready for the first new algorithm  which is called Hybrid A*. As name suggests, Hybrid A* isn't purely discrete or continuous. 
* Sampling based method called polynomial trajectory generation, which is really useful for highway driving. 


<img src="./outputs/tg1.png" height="500" width="500" />


Let's formally define the motion planning problem. 

### The Motion Planning Problem 


In the first lesson (Search), we learned some best planning algorithms, which solve the motion planning problem. But we never formally defined that problem. Let's do that now. Now, there is a word that we encounter quite a bitwhen we start reading material regarding motion planning algorithm. That word is **"configuration space"** which defines all the possible configurations of a robot in the given world. Consider the maze world we used in the first lesson where these worlds were all 2D grids, the robot configuration was sometimes two dimensional when we presented it as an x,y point, 

<img src="./outputs/tg2.png" height="300" width="300" />



and sometimes three dimensional when also including the robot's heading.


<img src="./outputs/tg3.png" height="300" width="300" />


In fact, the configuration space for vehicle can become even larger depending on what motion planning algorithms we decide to use. 


With this idea of configuration space in mind, we can define a motion planning problem as follows. We're given three things. An initial configuration, a goal configuration and also some constraints describing how the vehicle was allowed to move, its dynamics and the description of the environment. 

<img src="./outputs/tg4.png" height="600" width="600" />


Here, it is important to understand how this connects to the other decision-making modules that we have learned about. Usually, the start configuration is the current configuration of the car given to us by the localization value and the sensors that give us information about car location, speed, acceleration they can go, etcetera. The behavior layer gives us a desired end configuration, and maybe some constraints regarding where to go and at which speed. Finally, the prediction completes this problem by giving us information about how the obstacle region will evolve in time. This way, the sequence of actions that we generate takes into account other vehicles and pedestrian actions. And if we're working with a more sophisticated prediction module, how our actions might influence them. 

<img src="./outputs/tg5.png" height="600" width="600" />



The motion planning problem can then be defined as final sequence of feasible movements in the configuration space that's moved the robot from a start configuration to an end configuration without hitting any obstacles. 

### Properties Of Motion Planning Algorithm 


When discussing planning algorithms, there are two important properties that we like to talk about. 

* The first one is called **completeness**. 
    * And this means that if a solution exists to the motion planning problem, the planner will find it. 
    * And if a solution does not exist, the planner will terminate and report that no solution exists. 

So, consider the following two situations (in image). In the first one (left grid), the complete algorithm might correctly identify the marked path as a solution in map grid1 and the second one, the algorithm would terminates and tell us there is no solution. 

<img src="./outputs/tg6.png" height="600" width="600" />

Now, there may be something not right about the solution identified in the first example. Which brings us to the second property, **optimality**
    * A planning algorithm is optimal if it always return the sequence which minimizes some cost function. 
    * So if we're using total distance traveled as a cost function for example, then left would be an optimal solution while right would not. 

<img src="./outputs/tg7.png" height="600" width="600" />
    

### Types of Motion Planning Algorithms 



There are many classes of motion planning algorithms and we'll focus on one of these classes, but it's worth mentioning the others. 

* **Combinatorial methods** usually consists in dividing the free space into small pieces and first solving the motion planning problem by connecting these atomic elements. They are very intuitive ways to find initial approximate solution, but they usually do not scale well for large environments. 

<img src="./outputs/tg8.png" height="400" width="400" />



* Next, **potential fields** are reacting methods. Each obstacle is going to create a sort of anti-gravity field which makes it harder for the vehicle to come close to it. For example, you could imagine using this idea around pedestrians or bikes to encourage your planning algorithm to find trajectories that stay away from them. The main problem with most potential field methods is that they sometimes push us into local minima which can prevent us from finding a solution. 

<img src="./outputs/tg9.png" height="400" width="400" />


* **Optimal control** consists in trying to solve the motion planning problem and the controlling input generation in one algorithm. Using a dynamic model of a vehicle for start configuration and an end configuration, we want to generate a sequence of inputs,for example, steering angle and throttle inputs that would lead us from start to end configuration while optimizing a cost function relative to the control inputs such as minimizing gas consumption and relative to the configuration of the car, such as staying at a distance from other vehicles. There are a lot of very nice ways to do that. Most of them based on numerical optimization methods. However, it is hard to incorporate all of the constraints related to the other vehicles in a good enough way in order for these algorithms to work fast. 

* Finally, there are **sampling based methods**, which are what we will focus on today. 

<img src="./outputs/tg10.png" height="600" width="600" />



These algorithms are very popular because they require a somewhat easier to compute definition of the free space. Sampling based methods use a collision detection module that probes the free space to see if a configuration is in collision or not. Unlike combinatorial or optimal control methods, which analyzes the whole environment, not all parts of the free space need to be explored in order to find a solution. Explored parts are stored in a graph structure that can be searched with a graph search algorithm like Dijkstra or A star. Two main classes of methods can be identified as sampling based: discrete methods, which rely on a finite set of configurations and/or inputs, like a grid superposed on top of our configuration space, and probabilistic methods, which rely on the probabilistic sample of a continuous configuration space. The set of possible configurations or states that will be explored is potentially infinite, which gives some of these methods the nice property that they are probabilistically complete and sometimes probabilistically optimal meaning that they will always find a solution if you allow them enough computation time.

<img src="./outputs/tg11.png" height="600" width="600" />


We have barely scratched the surface on all the different sorts of planning algorithms that exist. Strongly encuraged to study the algorithms mentioned in the above image



### A\* reminder from Sebastian's Search Lesson


#### Artificial Intelligence for Robotics 

[Jupyter Notebook on Ghub Link](https://github.com/atul799/T3_Search/blob/master/Search.ipynb)


Transcript from Sebastian's Video:

Now I want to come with you to the absolute meat of this class, which is called A-star. A-star was invented by Nels Nelson at Stanford many years ago, and is a variant of the search algorithm that's more efficient than expanding every node. If you've gotten so far, and you understand the mechanism for searching by gradually expanding nodes in the open list, A-star is almost the same thing but not quite. To illustrate A-star I'm going to use the same grid as before but with a different obstacle configuration. This is oine way A-star performs really well. Obviously we are forced to go down to here, but in here we still have to search for the optimal path for the goal. Here is the same in problem code; you can see all the ones over here. Start set is over here, goal set is over here. If I run this code and give you my expand list, the ones you programmed before, you'll find that the expansion goes down from here, but then it expands into the open space. Diagonally it expands into the open space and until it finally hits the goal node 15. This took 16 expansions to get to this point. Let me now switch on A-star and run the code again. What we now find it only takes 10 expansions to get to this point, zero to nine over here. So it expands down to four, but then it expands straight toward the goal never touching this area over here, somehow magically knowing that up here the path to the goal will be longer than going straight. Now I didn't cheat. I didn't tell it that there's a straight path over here. So let me put an obstacle right here next to the goal and run A-star again. What you'll find it does expand up to seven over here but then moves to the second line over here, expands up here, and then hits the goal again. So it kind of does the minimum amount of work necessary to make maximum progress to the goal. That's A-star, and now we look into A-star in more detail. A-star uses a so called heuristic function, which is a function that has to be set up. If its all zeros then A-star resorts back to the search algorithm already implemented. If we call the heuristic function h, then for each cell it results into a value. So let me give you some values. Here is one: Its number of steps it takes to the goal if there was no obstacle. Clearly the number I'm putting in right now , 1, 2, 3, 4, 5, and so on, are not reflective of the actual distance to the goal because they don't consider the obstacles. In a world without obstacles the heuristic function that I'm giving you would actually measure the distance to the goal. So the heuristic function has to be an optimistic guess how far we are from the goal. So put differently, for any cell x y the heuristic function has to be an optimistic guess, which means a smaller equal to the actual goal distance from the coordinate x and y. Now that sounds a little bit ad hoc, but very often you can give good heuristic functions really easily like in this case over here. If we just know that the agent can move left, right, up, or down, it's really easy to say what is the number of steps it would take the agent with no obstacles to get to the goal location, and that's this table over here. That is easily generated automatically. Now in reality this is an underestimate. If obstacles, for example, look like this then from here it takes you more than 9 steps to get to the goal. It takes you 13 steps to over the hump over here. Therein lies the beauty of the heuristic function. It doesn't have to be accurate. If it was accurate you probably already solved the planning problem. There has to be a function that helps you understand where to search next in the case of ties, and it has to be just so that it underestimates or at best equals the true distance from the goal. Many, many problems have functions like these in our self-driving car. We use a function just like this; in fact the function I was just showing you, we are using in our software for free-form navigation. It boils down much to the number of which cell steps but for the Euclidean distance to a target location. I hope you understand how heuristic function might look like. It has many, many value heuristic function including setting everything to zero, which would not really help me. So let's work with this one heuristic function. Here is the heuristic function in the code. You can see the same heuristic function. The key modification now for our search algorithm is really, really simple. We again have an open list, and we add our state, we write down the g-value, but we also write down the g-value plus the heuristic value. G-value here is zero; heuristic value is 9. So the sum of the two is 9, and I call this the f-value. This is the cumulative g-value plus the heuristic value as looked up in the table over here. If I now expand I remove the element with the lowest f-value and not the lowest g-value. That's all there is to A-star. Let me give you an example. Say we went to the open list all the way down here. That is we expanded all these states over here, and this is the one present here on the open list. Our g-value will be 5. Our heuristic will be 4, and the sum is 9 as before. Let's now expand this node. We get to this one over here, the g-value increases to 6. G plus heuristic is still 9. Now let's expand it more, and there's now two options finally: This state over here and this state over here. The one up here is called 3 2, the one on the right is called 4 3. The g-value over here in both cases is 7, but when we add the h-value we get a difference. Up here we find the h-value to be 4. We kind of moved a little bit away from the goal according to the heuristic. That gives us a total of 11. Whereas for the feed over here we find the h-value to be 2. 100 Here is the first time that A-star makes an actual difference. 101 It has a preference to expand this node over here over the node over here. 102 To see why the f-value, the sum of g and h, over here is 9 but over here is 11. 103 What this reflects is that, according to the heuristic, 104 this guy is actually 2 steps closer to the goal than this guy over here. 105 This guy, according to the heuristic, may be 2 steps away from the goal, 106 and the guy over here is at least 4 steps away. 107 A-star now will expand this node over here because its f-value is 9 versus 11. 108 So let's do this. 109 In expanding this node we find there is two valid neighbors: 110 the guy up here and the guy on the right. 111 The first guy's coordinate are 3 3. 112 The second guy is 4 4. 113 As before we increment the g-value by one. 114 It was eight in both cases. 115 Now we add the heuristic to the g-value, which for the first one over here is 3; 116 Whereas for the one on the right we get one as the heuristic. 117 That's the result of expanding the node over here. 118 Here is our new open list, and again we have a preference. 119 On the open list are these three states, and we prefer the one on the right 120 because its f-value is smaller than the other two f-values. 121 The one over here is 9; the ones over here have an f-value of 11. 122 So once again we expand, and in the expansion will be the goal state, 123 and then we find the goal set and we're done without ever expanding anything in the maze up here. 124 That feels like magic, but the key thing here is by providing additional information, 125 the so called heuristic function, we can guide the search. 126 When we have an impasse we can pick a node that looks closer to the goal state. 127 As a result we will likely make more progress towards the goal. 

### A\* Properties

* It uses an optimistic heuristic function to guide grid cell expansion.

* It always finds a solution if one exists (completeness)

* Solutions it finds are always optimal assuming an admissible heuristic


<img src="./outputs/tg12.png" width="600" height="600" />

The first step before running A\* is defining a discrete set of states that represent the world we want to plan in.  A\* is a discrete method. 
 
In order to be a valid, A* heuristic have to be optimistic. Meaning that they should underestimate the actual cost to go from the cell to the goal. 

A\* does always find the solution if one exists. Which as you recall meaning that A* is a complete planning algorithm. 

So, the next assertion is somewhat tricky because it depends on the search space that is being used and what it represents. If the search space represent only the position x, y, and we're trying to move a car, there is no guarantees that the sequence of position returned by A\* will be drivable.  However, remember that A\* is the graph search algorithm. We can define a more complex search space including heading for example and restrict connections between cells such that when we close a cell we add to the open set only cells such that the transition from the closing cell to the new cells are derivable. If we do that the resulting path will be derivable as well. This corresponds to running A\* on the lattice. 

It is true that the solution A\* finds are always optimal with respect to the discretization of the environment that was used as an input. The more granular the discretization, the better the solution. We call this resolution optimal. 

## Hybrid A\* Introduction 


The next algorithm we are going to learn is called Hybrid A\*. To introduce to Hybrid A\*, let's see a video that Sebastian recorded for another class. 


So let's talk about robot path planning or robot motion planning. Where just a rich field in itself and I can't give you a complete survey of all the items involved. But one of the key differences to the planning as we talked about before is that now the word is continuous. So for example, we've read about A* in which we discretize the world. You might have a goal location, you might have obstacles and then A*, a valid action sequence might look like this.

<img src="./outputs/tg13.png" width="300" height="300" />


And even though this is a valid solution to the planning problem, a car can't really follow these discrete choices. There's a number of very sharp turns over here that are just irreconcilable with the motion of a car. So the fundamental problem here is **A\* is discrete** whereas the **robotic world is continuous**. 



So the question arises, is there version of A\* that can deal with the continuous nature and give us provably executable paths? This is a big big question in robot motion planning. And let me just discuss it for this one example and show you what we've done to solve this problem in DARPA urban challenge. 

So the key to solving this with A\* has to do with a state transition function. Suppose we are in a cell like this and you play a sequence of very small steps simulations using our continuous math from before, then a state over here might find itself right here in the corner of the next discrete state. Instead of assigning this just to the grid cell, an algorithm called Hybrid A\* memorizes the exact $ X^{'} , Y^{'}, theta^{'} $ and associate it with the grid cell over here the first time the grid cell is expanded. Then when expanding from the cell, it uses a specific starting point over here to figure out what the next cell might be. 
<img src="./outputs/tg14.png" width="300" height="300" />


Now, it might happen that the same cell we used to get in A\* maybe from over here going into a different continuous expansion of X, Y and theta, but because in A\* we tend to expand cells along the shortest path before we look the longer paths, we not just cut this off and never consider to stay over here again. 
<img src="./outputs/tg15.png" width="300" height="300" />


Now, this leads to a lack of completeness which means there might be solutions to the navigation problem that this algorithm doesn't capture. But it does give us correctness. So as long as our motion equations are correct, the resulting paths can be executed. Now, here's a caveat. The motion equations  are an approximation and it's only correct to the point that these motions equations are correct or not correct. But nevertheless, all paths that come out are nice, smooth, and curved paths. And every time we expand the grid cell, we memorize explicitly the continuous values of $ X^{'} , Y^{'}, theta^{'} $ with this grid cell. 

## Hybrid A\* Tradeoffs 




Now, unlike regular A\*, Hybrid A\* is a continuous method. Since, the configuration that weill end up being part of the trajectory are not part of a predefined discrete set. 

Similarly to A\*, Hybrid A\* star also uses an optimic heuristic. 

But one sacrifice made with Hybrid A\* is that it does not always find a solution when one exists. Indeed, the completeness is sacrificed because we only allow one continuous configuration for discrete A\*. 

The great improvement compared with A\* is that the solution it does find are guaranteed to be driveable though not necessarily optimal compared to the properties of A\*. 

With Hybrid A\*, we have less completeness and optimality guarantees in favor of driveability. In practice, however, this algorithm is very efficient at finding good path almost all the time.


<img src="./outputs/tg16.png" width="600" height="600" />

## Hybrid A\* In Practice


In the first video about Hybrid A\*, the update equations that were used were somewhat generic to most X,Y,theta configuration spaces. In particular, we don't specify what Omega is. Omega is the heading rate of change. Now for some robots, it might be that we can specify any omega independently of its state. But that would mean that the robot can turn around its Z axis without constraints. For car though, this is not very realistic, and we should probably use the bicycle model. This gives us an equation for Omega like shown in image. Here, 'L' is the distance between the front and rear axle, $ \Delta $ is the steering angle and v is the constant positive velocity. 
<img src="./outputs/tg17.png" width="600" height="600" />

Now, for our Hybrid A\* algorithm, we need to decide which configuration can be reached from the current node considered, in order to add them to the open set. In practice, we would probably use some number of steering angles between maximum left steer and maximum right steer angle. But the more different Deltas you add the longer it takes for your algorithm to complete. Therefore, in this example, we will only choose three angles to pick delta from; the maximum left steering angle, zero steering angle and the maximum right steering angle. In the case of a car that can turn its wheels 35 degrees at most, that would be -35, 0 and 35. This allows us to expand the search tree with three new motion primitives; go straight, steer left and steer right.
<img src="./outputs/tg18.png" width="600" height="600" />




Let's see how that works in a robot maze. Let's say we have a maze like this. The red square is the goal and this is the start position. The first thing we would do is add this cell to open list. Use gray to indicate that.
<img src="./outputs/tg19.png" width="300" height="300" />


And then we would generate three trajectories, the one for going straight, the one for turning right, the one for turning left.
<img src="./outputs/tg20.png" width="300" height="300" />

And in this case, we can discard the trajectories that go off the map or collide. This gives us one new cell to open, which we do, and then we mark the first cell closed which is indicated with check-mark. 
<img src="./outputs/tg21.png" width="300" height="300" />


Now, there is only one open cell so I choose to expend that one next. And again, we generate the three trajectories from this stage, one of them is infeasible, so let's not keep looking at it. And this leaves us with two new cells to open.  close the previous. Now there are two choices of what cell to expand. 
<img src="./outputs/tg22.png" width="300" height="300" />

Assuming we have some heuristic that values being closer to the goal, we choose the cell on the right,
<img src="./outputs/tg23.png" width="300" height="300" />


and then generate trajectories from this new state. Note, how, in all these steps, we are keeping track of, not just what cell we am in, but also where in that cell we are, including orientation. 
<img src="./outputs/tg24.png" width="300" height="300" />

In practice, we could also use a three state space that include **heading**. If we don't do that, then we're likely to prematurely close cells. We are not doing it that way here, mostly because it's harder to visualize. 

Let's keep going. We have three new cells to open. We have to close the previous cell, find the best option in the set of open cells and repeat. 
<img src="./outputs/tg25.png" width="300" height="300" />


Again leaving us with three new cells to add to the open list. Which means we can close the the previous cells and things get a little more interesting here. The heuristic will probably suggest that we expand the lower right cell and generate trajectories from there. But this time, all of them would collide with the wall.
<img src="./outputs/tg26.png" width="300" height="300" />

Therefore, we don't need to add any new cell to the open set. Now, since we did expand this cell we have to mark it closed,  
<img src="./outputs/tg27.png" width="300" height="300" />

and then we can look at my remaining open cells.


So same as before, we take a closest cell to the goal, generate trajectories, that is, that all of these collides and close that cell. 
<img src="./outputs/tg28.png" width="300" height="300" />

Of the remaining open cells, this upper right one is closest or the one with the best heuristic.
<img src="./outputs/tg29.png" width="300" height="300" />


So we repeat the process, on and on, until we reach the goal. 
<img src="./outputs/tg30.png" width="300" height="300" />



Now, let's look at what this trajectory really looks like by getting rid of some of the clutter. And what we're left with is a continuous path, that might not be the smoothest solution to our planning problem. 
<img src="./outputs/tg31.png" width="300" height="300" />

Indeed we can easily find examples of environment, where the resulting path doesn't look very good compared to what human drivers would have done in this situation. And that's not the only problem. Sometimes, everyday stuff fails to even find the solution when one actually exists. For example, let's go back to the stage in the algorithm 
<img src="./outputs/tg32.png" width="300" height="300" />
and consider what would happen if the map were slightly changed. 

<img src="./outputs/tg33.png" width="300" height="300" />

In this case, we would have made all the same decisions up until this point. But now we have close this crucial cell. Clearly, there is a path to the goal that could be reached. But in these circumstances, Hybrid A\* will not find any. We can reduce the likelihood of this problem by increasing the resolution of our grid or adding a third dimension to the search space for the heading. Indeed, this would allow us not to close this position in the map for all headings but only for a specific range which may allow us to find the solution in this- 

## Hybrid A\* Heuristics

The paper [Junior: The Stanford Entry in the Urban Challenge](junior-the-stanford-entry-in-the-urban-challenge.pdf) is a good read overall, but Section 6.3 - Free Form Navigation is especially good and goes into detail about how the Stanford team thought about heuristics for their Hybrid A\* algorithm (which they tended to use in parking lots).

Read section 6.3 of the paper and then answer the following question:

### Quiz: Valid Heuristics
If you have two individually admissible heuristic functions $h_1$ and $h_2$, which of the following combinations are also valid?
* h1+h2
* $\frac{h1+h2}{2}$
* min(h1,h2)
* max(h1,h2)   <--

we need to guarantee that our combination underestimates the actual cost to the goal


## Hybrid A\* Pseudocode:
The pseudocode below outlines an implementation of the A\* search algorithm using the bicycle model. The following variables and objects are used in the code but not defined there:

* <span style="color:red">State(x, y, theta, g, f)</span>: An object which stores <span style="color:red">x, y</span> coordinates, direction <span style="color:red">theta</span>, and current <span style="color:red">g and f</span> values. grid: A 2D array of 0s and 1s indicating the area to be searched. 1s correspond to obstacles, and 0s correspond to free space.
* <span style="color:red">SPEED</span>: The speed of the vehicle used in the bicycle model.
* <span style="color:red">LENGTH</span>: The length of the vehicle used in the bicycle model.
* <span style="color:red">NUM_THETA_CELLS</span>: The number of cells a circle is divided into. This is used in keeping track of which States we have visited already.


The bulk of the hybrid A\* algorithm is contained within the <span style="color:red">search</span> function. The <span style="color:red">expand</span> function takes a state and goal as inputs and returns a list of possible next states for a range of steering angles. This function contains the implementation of the bicycle model and the call to the A\* heuristic function.
```python
    def expand(state, goal):
        next_states = []
        for delta in range(-35, 40, 5): 
            # Create a trajectory with delta as the steering angle using the bicycle model:

            # ---Begin bicycle model---
            delta_rad = deg_to_rad(delta)
            omega = SPEED/LENGTH * tan(delta_rad)
            next_x = state.x + SPEED * cos(theta)
            next_y = state.y + SPEED * sin(theta)
            next_theta = normalize(state.theta + omega)
            # ---End bicycle model-----
        
            next_g = state.g + 1
            next_f = next_g + heuristic(next_x, next_y, goal)
    
            # Create a new State object with all of the "next" values.
            state = State(next_x, next_y, next_theta, next_g, next_f)
            next_states.append(state)

        return next_states

    def search(grid, start, goal):
        # The opened array keeps track of the stack of States objects we are 
        # searching through.
        opened = []
        # 3D array of zeros with dimensions:
        # (NUM_THETA_CELLS, grid x size, grid y size).
        closed = [[[0 for x in range(grid[0])] for y in range(len(grid))] for cell in range(NUM_THETA_CELLS)]
        # 3D array with same dimensions. Will be filled with State() objects to keep 
        # track of the path through the grid. 
        came_from = [[[0 for x in range(grid[0])] for y in range(len(grid))] for cell in range(NUM_THETA_CELLS)]

        # Create new state object to start the search with.
        x = start.x
        y = start.y
        theta = start.theta
        g = 0
        f = heuristic(start.x, start.y, goal)
        state = State(x, y, theta, 0, f)
        opened.append(state)

        # The range from 0 to 2pi has been discretized into NUM_THETA_CELLS cells. 
        # Here, theta_to_stack_number returns the cell that theta belongs to. 
        # Smaller thetas (close to 0 when normalized  into the range from 0 to 2pi) 
        # have lower stack numbers, and larger thetas (close to 2pi whe normalized)
        # have larger stack numbers.
        stack_number = theta_to_stack_number(state.theta)
        closed[stack_number][index(state.x)][index(state.y)] = 1

        # Store our starting state. For other states, we will store the previous state 
        # in the path, but the starting state has no previous.
        came_from[stack_number][index(state.x)][index(state.y)] = state

        # While there are still states to explore:
        while opened:
            # Sort the states by f-value and start search using the state with the 
            # lowest f-value. This is crucial to the A* algorithm; the f-value 
            # improves search efficiency by indicating where to look first.
            opened.sort(key=lambda state:state.f)
            current = opened.pop(0)

            # Check if the x and y coordinates are in the same grid cell as the goal. 
            # (Note: The idx function returns the grid index for a given coordinate.)
            if (idx(current.x) == goal[0]) and (idx(current.y) == goal.y):
                # If so, the trajectory has reached the goal.
                return path

            # Otherwise, expand the current state to get a list of possible next states.
            next_states = expand(current, goal)
            for next_state in next_states:
                # If we have expanded outside the grid, skip this next_state.
                if next_states is not in the grid:
                    continue
                # Otherwise, check that we haven't already visited this cell and
                # that there is not an obstacle in the grid there.
                stack_number = theta_to_stack_number(next_state.theta)
                if closed_value[stack_number][idx(next_state.x)][idx(next_state.y)] == 0 and grid[idx(next_state.x)][idx(next_state.y)] == 0:
                    # The state can be added to the opened stack.
                    opened.append(next_state)
                    # The stack_number, idx(next_state.x), idx(next_state.y) tuple 
                    # has now been visited, so it can be closed.
                    closed[stack_number][idx(next_state.x)][idx(next_state.y)] = 1
                    # The next_state came from the current state, and that is recorded.
                    came_from[stack_number][idx(next_state.x)][idx(next_state.y)] = current
```

## Implementing Hybrid A\*

In this exercise, a working implementation of a <span style="color:red">breadth first search</span> algorithm which does not use any heuristics to improve its efficiency is given. The goal is to try to make the appropriate modifications to the algorithm so that it takes advantage of heuristic functions (possibly the ones mentioned in the previous paper) to reduce the number of grid cell expansions required.

Instructions:
* Modify the code in 'hybrid_breadth_first.cpp' and hit <span style="color:red">Test Run</span> to check your results.
* Note the number of expansions required to solve an empty 15x15 grid (it should be about 18,000!). 

Refer to [source code](./src/hybrid_breadth_first.cpp)


The implementation is using the "distance to goal" heuristic function. In this implementation, we have added an f value to the maze_s struct, which is set in the expand function. Additionally, we've added two new functions: heuristic and compare_maze_s. The compare_maze_s function is used for comparison of maze_s objects when sorting the opened stack.

To get an even lower number of expansions, try reducing NUM_THETA_CELLS in hybrid_breadth_first.h to reduce the total number of cells that can be searched in the closed array. Be careful though! Making NUM_THETA_CELLS too small might result in the algorithm being unable to find a path through the maze.

Another possibility for improvement is to use the regular A\* algorithm to assign a cost value to each grid cell. This grid of costs can then be used as the heuristic function, which will lead to an extremely efficient search!

## EnvironmentClassification


Hybrid A\* is one of the best algorithm for finding trajectories in unstructured environments. 
An example of such an environment is a parking lot or a maze. And these environments tend to have 
* less specific rules than highway or streets 
* lower driving speeds. 
* They also do not have an obvious reference path that corresponds to what we should be driving 90% of the time, because they change so much. 

<img src="./outputs/tg34.png" width="300" height="300" />


On the other hand, highway or streets are very structured environments and in these environments, 
* all motion is more constrained by predefined rules, regarding how we can move on the road. 
    * For example, the direction of traffic or lane boundaries or also speed limits. 

<img src="./outputs/tg35.png" width="300" height="300" />

All these rules impose constraints which have to be satisfied but also for guidance regarding what the trajectory should look like. And while A\* is great at finding solutions everywhere, it does not take advantage of this information. For example, the road structure itself can be used as a reference path. 

## Frenet Reminder 

We'll be relying heavily on Frenet coordinates. And as a reminder of how these work, we can first look at the road in Cartesian coordinates, where we imagine the road leaving on some rectangular X, Y grid. And you can imagine that the more complex the road shape, the harder our problem. Think of how complicated both the X and Y behavior would look like just to describe a car going along the lane. If we were to compute that trajectory, we would pretty much be computing the shape of the road whereas this information is already available. So instead we use a Frenet coordinate system where the S coordinate depicts motion along the road also called longitudinal motion and the D coordinate gives side to side or lateral motion. We can simply take the center of a lane as a reference line, we could also imagine using an actual nice and feasible trajectory computed with enough lane planning algorithm like A\* or Hybrid A\*. And that trajectory would have been later smooth. The benefit of this is that it would allow us to have a nominal path that we know the vehicle should follow if possible, like when the corresponding portion of the lane in front of us is empty. 

<img src="./outputs/tg36.png" width="500" height="500" />

## The Need For Time


It is important to not just compute a sequence of configurations, but also to decide when we're going to be in each configuration. So let's say We are told that some car recently drove along this road following this trajectory. If we asked to describe what the driver did, one might be tempted to say that they **passed the slow moving vehicle** or maybe, they think that it **swerved to avoid a pothole**. 

<img src="./outputs/tg37.png" width="300" height="300" />


Well, what actually happened up here, was that a broken down vehicle stopped on the road, and down here we have a quickly moving vehicle that will travel the trajectory we just saw. Now, let's track the location of this vehicle with little blue dots. Add one here for T equals zero. In the next time step, the vehicle has moved forward quite a bit and again and again, but this time not as much because it started to slow down and slowing down again. At this point, this vehicle would like to change lane in order to pass the stopped vehicle. However, at the same moment it senses a vehicle arriving behind it in the other lane at high speed. Therefore, it has to wait until this new car has passed it in order to change lane which only gives it the option to come to a full stop and wait. T=7, T=8 and T=9 when the red vehicle has passed it. The blue vehicle decide that it's safe to keep driving. It does this by turning left and accelerating. It continues with the lane change maneuver, until it gets into the left lane where it keeps accelerating and accelerating until it gets to highway speed. Once it has passed the stop vehicle, it gets back into the right lane and continues on its way. 

![pic](./outputs/tg38.gif)


This story was told using discrete time increments, but the vehicle was following a continuous path that looked something like this. 

<img src="./outputs/tg39.png" width="300" height="300" />

If we remove the dots, we're left with the same path as in the beginning. The problem was not incorporating timing into our planning, is that you don't tell the full story. 
You describe the motion of your vehicle in the next few seconds, but you don't actually specify where the vehicle should be at a given time within this time frame. When we were solving static Mazes problems this was fine but highways aren't Mazes. There is traffic on highways which means that the traversal space is constantly changing over time. To avoid collision in dynamic environment, we need to think about time as well. And that means that driving in traffic is fundamentally a three dimensional problem. It is important to note that you have the same situation if you were using Cartesian coordinates. Frenet coordinates help us simplify the problem, but do not change the nature of the problem, which is planning a trajectory in dynamic environment. 

<img src="./outputs/tg40.png" width="500" height="500" />

### S D And T 


Up until now we learned about generating a sequence of configurations for vehicle using algorithms like A\* and Hybrid A\*. We refer to this as path planning. But what about time? In path planning, we imagine taking our driving surface and length some sort of discretized grid on top of it. Then, we make plans by specifying a start state and a goal state. And the trajectories we generate looks something like this for algorithms like A\* or something like this for algorithms like Hybrid A\*.

<img src="./outputs/tg41.png" width="500" height="500" />


These algorithms play an important role in autonomous driving. But the path they generate don't really take predictions into account. For example, let's project back to when the blue vehicle first wanted to change lane. Here we will include the time as a third dimension. One way to visualize it is to imagine looking at the road from some angle and time would be the dimension going up. 
<img src="./outputs/tg42.png" width="400" height="400" />


And one way for a vehicle to move along the path that goes around the start vehicle, would be something like that. 
<img src="./outputs/tg43.png" width="400" height="400" />



The fast moving vehicle we perceived behind us is predicted to move in on the lane at a fast speed which would look something like that. 

<img src="./outputs/tg44.png" width="400" height="400" />

And that's how we know that we cannot change lane immediately. If we try to go before the vehicle that passed us, which is  red paralellogram in the two dimension. We will collide with the vehicle. We have to wait for this vehicle to drive past us. Once the Red car has passed us, we can proceed with our lane change. 
<img src="./outputs/tg45.png" width="400" height="400" />

Now as we can see, having a path is nice, but it is not sufficient to solve the planning problem when there are dynamic objects around us which in autonomous driving is all the time. Therefore, we have to plan not only a sequence of configuration, s,d and theta but also have these configurations all set up in time. The trajectory is a function which for any time t associates a configuration of a vehicle. 

<img src="./outputs/tg46.png" width="400" height="400" />

Now, we need a way to generate such a trajectory. And the way we're going to do this, is by separating the planning we do in the "s" dimension, from the planning we do in a "d" dimension. Instead of thinking about one picture in three dimension, we usually decide to reason about two pictures. Each of which is in two dimensions. 
Let's walk through the same vehicle tracking exercise we did in the previous video, two more times. One for s and one for d. 

Let's start with s. At t=0. The vehicle is at s=0. Then at t=1, it's at s=1. And it keeps moving until it reaches the situation where it goes to a full stop. Note that we can see the vehicle has stopped by looking at this graph. Here we can see a flat slope and the flat slopes means no movement. This is something we couldn't see in that t over s graph that we used before to represent a path. Then the vehicle starts moving again until it reaches its final configuration at t=15. 

<img src="./outputs/tg47.png" width="400" height="400" />

Now, let's do the same thing for d versus t. As we move along the road, d remains constant. Then the vehicle goes to a full stop and after the stop, start shifting left. And after it has passed the green vehicle, it goes back to its original lane. 

<img src="./outputs/tg48.png" width="400" height="400" />

Finally, we can look at both trajectories simultaneously and the great thing is that we can finally visualize the whole motion of the car over time. It's not just a ***path*** now, it's a **trajectory** and we can fully describe what the vehicle did within these 15 seconds. 

<img src="./outputs/tg49.png" width="400" height="400" />

### Trajectory Matching
Here are some trajectory and actions associated:

<img src="./outputs/tg50.png" width="400" height="400" />  **Slow down in lane**

<img src="./outputs/tg51.png" width="400" height="400" />  **Pass vehicle**

<img src="./outputs/tg52.png" width="400" height="400" />  **Swerve quickly **

<img src="./outputs/tg53.png" width="400" height="400" />  **Pull over and stop** 


## Structures Trajectory Generation Overview 

In the next few segments, We are going to learn a method for trajectory planning that is useful in structured environments like highway driving. To do that, we get introduced to the idea of Jerk minimization to generate a nice trajectory from a start configuration to a goal configuration. Lets see how to generate Jerk minimizing trajectories using polynomials and then present a brief derivation on how to compute the coefficients of the polynomial that solves our problem. Then we see an example of what one such trajectory looks like on the highway and how we can evaluate its drive-ability and compare it to other trajectories. Finally, we see how by generating many similar jerk minimizing trajectories we can compare them and select the best one for the situation we want to drive through.


<img src="./outputs/tg54.png" width="600" height="600" />

## Trajectories With Boundary Conditions 

Our goal is to generate continuous trajectories, but how do you do that? Let's start with a naive approach and then gets more realistic. Let's say, our car has been driving along the road at a constant speed for some time. The s versus time graph is just a straight line, since the velocity is constant, and the differences time graph is flat, since the car is just staying in the center of the lane. Also note that we've shifted the axis a bit so that the origin of the frame is our current position is in s,d and t. 

<img src="./outputs/tg55.png" width="600" height="600" />


This will end up being computationally efficient layer. Now let's say we're getting off this highway soon. So we want to change lane in order to end up there. After some delta-T which let's say it's 10 second. On the s,d and t-graph that goal would be here and here. 

<img src="./outputs/tg56.png" width="600" height="600" />

The stop position and go position are fixed. They define what we call the boundary conditions of our trajectory at t equals zero, and t equals ten. If those were our only boundary conditions, then we might consider s and t-trajectory is looking like this. But it turns out that these aren't physically possible. 
<img src="./outputs/tg57.png" width="600" height="600" />

This kink in the slope would translate in an instantaneous jumping speed which would require infinite acceleration. Now, if we were to send that to our motion control module, maybe they can track this trajectory, but probably not. And even if they tried to track it, it would result in a very strong acceleration of our car, which is both uncomfortable and dangerous. 

This is why we need both continuity and smoothness in our trajectories. But how much continuity and smoothness? We know we want continuity in position, since our car cannot teleport. And it would not make sense to compute a trajectory starting 10 metres ahead of us. And we also know that our car cannot instantaneously change speed, therefore the speed should also be continuous. Following this logic you could also say that the acceleration should be continuous and maybe the acceleration's derivity as well and so on. Position becomes velocity, velocity becomes acceleration, acceleration becomes jerk, and going even further you get snap, crackle, and pop. It turns out that one of these quantities is directly related to human perception of comfort. So let's think about it. In a vehicle, humans feel discomfort when which of these values is high? Velocity? No. When they cruise down the highway at high velocity, people usually don't find it less comfortable than when they drive at lower speed. And if you think about airplanes, they move very fast and they aren't really uncomfortable. Acceleration maybe? That's slightly more tricky. We are indeed sensitive to acceleration but mostly when it becomes very high. For example, we are all used to supporting acceleration up to one G which is the most you can usually sustain in a normal car. Therefore, we shouldn't reach acceleration levels that are high enough to become uncomfortable. Jerk, then? It turns out, this is the quantity that humans perceive as uncomfortable. We can tolerate high acceleration but we don't like it when our acceleration changes too quickly which is high jerk. So when we design trajectories for self-driving car, we would like it, if they were jerk minimizing

<img src="./outputs/tg58.png" width="600" height="600" />


## Jerk Minimizing Trajectories 


What is a jerk minimizing trajectory? An interesting and useful fact to us is that it is relatively easy to compute a jerk optimal trajectory in one dimension. In fact, consider a function s of t different from time zero to time tf. and recall that jerk is a third relative of position. So the total jerk accumulated over the duration of this trajectory is given by this equation. And here, we want to analyze both positive and negative jerk. Therefore, we're going to look at total square jerk. Our problem is then to find a function s of t that minimizes this integral. 

<img src="./outputs/tg59.png" width="300" height="300" />

If we go through the math required to find the minimum, you'll find that all the time derivatives of s are further than six and more have to be zero in order for S to be jerk minimal. Now, we can also use a helpful fact about functions, which is that any function can be written like this or more compactly like this. When you take this form of s and add in the information that we have about it's time derivative, you find this. All of the coefficient bigger than five or zero, which means that all minimum jerk trajectories can be represented as a fifth order polynomial like this. 
<img src="./outputs/tg60.png" width="500" height="500" />

You can see this equation has six coefficients and six coefficients means six tuneable parameters that we can choose from in order to define the shape of a 1D trajectory. And in practice, we use them to specify the boundary conditions of our trajectory. The things we'd like to constrain are the initial position, velocity, and acceleration as well as the final position, velocity and acceleration. Now, this is just the 1D representing the longitude of displacement. But, the same for the lateral displacement applies. This gives us 12 variables to pick in order to fully define the motion of our vehicle in S&D over time.
<img src="./outputs/tg61.png" width="500" height="500" />

The initial state of our vehicle give us the longitudinal and lateral boundary conditions at t equals zero, and we get to pick the end boundary condition. Let's see plots of what lateral trajectories look like for a simple case, where a car moves from here to here 
<img src="./outputs/tg62.png" width="300" height="300" />
and make the simplifying assumption that the initial and final lateral velocities and accelerations are zero.Which is reasonable for lane change. Then, all boundary conditions look like this. 
<img src="./outputs/tg63.png" width="400" height="400" />

**Sf should be 30 in above image**

And the only nonzero quantity of this 1D problem is the final lateral displacement.

Let's look at what some of these trajectories look like. Keep in mind, this is a one dimensional motion. The X axis here is time and the Y axis is lateral position of the vehicle as it makes its lane change. The F and T are the final lateral displacements and the duration of our trajectory. You can see at equal zero, the position is zero. The slope is zero so velocity is now 0 and so is acceleration. Likewise, for the final position that equals 10. And this curves looks pretty smooth. But let's also look at velocity over time and acceleration over time. Although not all of these curves might be meaningful to you, let's think back about what just happened here. We've specified a position we wanted to reach and a time we wanted to reach it at in some equations and we end up with a nice looking trajectory. It's very nice indeed because a lot of drivers would not attain that level of comfort if we were to ask them to reproduce it. 
<img src="./outputs/tg64.png" width="400" height="400" />

Now, let's see what happens if I try to reach the destinations more quickly by turning down T. Sure enough, we can still find the trajectory over time, but it's much more jerky. It is important to realize that although the trajectory that we generate using this method are jerk optimal for a given set of boundary conditions, they still depend heavily on these conditions.
<img src="./outputs/tg65.png" width="400" height="400" />

Therefore, it would be important in the future to wisely pick these end conditions. Now, let's find out how to compute what the six coefficient had to be in order to satisfy these boundary conditions. 

## Derivation Overview 

In the previous video, we saw that the position of a Jerk minimizing trajectory is given by quintic polynomial. In this section, let's see a quick and non-rigorous walk-through of how we find the six coefficients that describe this polynomial. It is not essential that you remember the details of this, but you might find it interesting. 

First, we differentiate the equations of the position in order to get the equations for the velocity. And then, we differentiate it again in order to get the equation for the acceleration. Now, we could just plug in our six boundary conditions to get the six equations. But first, we're going to do something that will make our life a little bit easier. We're going to set the initial time as zero. And when we do that, we find this, which means that three of four unknowns don't need to be identified anymore, which reduces the problem from six unknowns to three.
<img src="./outputs/tg66.png" width="400" height="400" />

And it simplifies those three trajectories by removing three of the unknowns here. We can gather the known terms into functions of the start boundary conditions to make it look like a bit cleaner. The C1, C2, and C3 terms are functions of the initial conditions. 
<img src="./outputs/tg67.png" width="400" height="400" />

What remains is that we only have to identify three parameters that depend only on our end boundary condition. And these equations look like this. And we know the final position, velocity, and acceleration at time TF since those are all the quantities that we picked. 
<img src="./outputs/tg68.png" width="400" height="400" />

Since we know them, a more useful way to think of these equations is like this. Where we have deliberately separated the known quantities from the unknown variables that we wish to determine. And this is starting to look like a very solvable system of three equations. The best way to solve this is with a matrix that looks like this. 
<img src="./outputs/tg69.png" width="400" height="400" />

And this problem can be solved by inverting this matrix using any matrix mathematical library which we'll do later. 

## How Polynomial Trajectory Generation Works 

Let' see how polynomial solver to generate Jerk minimizing trajectories being used. Let's consider the S coordinates first. As an input, it takes the current vehicle state, the goal state, and the duration. And as an output, it will generate six coefficients, which uniquely define the polynomial that describes the longitudinal trajectory. Similarly, we feed input parameters to our solver regarding lateral configurations and compute the lateral trajectory.
<img src="./outputs/tg70.png" width="600" height="600" />


We can use this approach in a situation like this one, you have traffic and a self-driving car. And let's say that the requested behavior is to pass the vehicle in front of us. With polynomial trajectory generation, we take the current configuration of the car along with its velocity and acceleration as our start state. And then we specify a valid goal state that leads our vehicle in the other lane. We ship these states into our polynomial solver along with the desired duration that will allow for the maneuver. And we get a Jerk minimizing trajectory to the goal. 

<img src="./outputs/tg71.png" width="500" height="500" />

## Implement a Quintic Polynomial Solver

In this exercise we will implement a quintic polynomial solver. This will let you take boundary conditions as input and generate a polynomial trajectory which matches those conditions with minimal jerk.

Inputs
Your solver will take three inputs.

* <span style="color:red" >start</span> - $ \begin{bmatrix}
    s_i & \dot{s_i} &  \ddot{s_i}
    \end{bmatrix} $
* <span style="color:red" >end</span> - $ \begin{bmatrix}
    s_f & \dot{s_f} &  \ddot{s_f}
    \end{bmatrix} $
* <span style="color:red" >T</span> - the duration of maneuver in seconds.




### Tips
Remember, you are solving a system of equations: matrices will be helpful! The Eigen library used from Sensor Fusion should be included.

* The equations for position, velocity, and acceleration are given by:

$ s(t)=s_i	+ \dot{s_i}*t + \frac{\ddot{s_i}}{2}*t^2 + \alpha_{3}*t^3 + \alpha_{4}*t^4 + \alpha_{5}*t^5 $

$ \dot{s}(t) = \dot{s_i} + \ddot{s_i}*t + 3 * \alpha_{3}*t^2 + 4 * \alpha_{4}*t^3 + 5 * \alpha_{5}*t^4 $ 

$\ddot{s}(t) = \ddot{s_i} + 6 * \alpha_{3}*t + 12 * \alpha_{4}*t^2 + 20 * \alpha_{5}*t^3 $

and if you evaluate these at t=0 you find the first three coeffecients of your JMT are:

$ \begin{bmatrix}
    \alpha_0 & \alpha_1 &  \alpha_2
    \end{bmatrix} $ = $ \begin{bmatrix}
    s_i & \dot{s_i} &  \frac{1}{2} * \ddot{s_i}
    \end{bmatrix} $

and you can get the last three coefficients by evaluating these equations at $t = T$. When you carry out the math and write the problem in matrix form you get the following:
<img src="./outputs/tg72.gif" width="500" height="500" />

All these quantities are known except for $\alpha_3, \alpha_4, \alpha_5 $

### Instructions
See the <span style="color:red" >JMT(start, end, T)</span> function in <span style="color:red" >[JMT_polynomial.cpp](./src/JMT_polynomial.cpp)</span>


## What should be checked?

When evaluating the feasibility of a potential trajectory, the following quantities should be checked

* Maximum velocity (with respect to car's capabilities and speed limit)

* Minimum velocity

* Maximum acceleration

* Minimum acceleration

* Steering angle


All of these qualities should be checked. Clearly, a car can only go so fast and speeds above the vehicle maximum are not feasible. But remember that minimum velocity corresponds to going backwards, and that matters, too. On a lot of roads, it's illegal to go backwards. For maximum acceleration, we need to consider both lateral and longitudinal acceleration. Maximum lateral accelerations need to be checked to avoid rollover or skidding. And maximum longitudinal acceleration depends on your vehicle's powertrain. Minimum accelerations would be negative and correspond to the vehicle maximum braking force. Also, your car can only turn so much, and steering angle should be checked as well.

<img src="./outputs/tg73.png" width="500" height="500" />


### Implementing Feasibility 


Here we're not going to discuss how to solve these feasibility check exactly. But we will get a few hints about how to do some initial validation for your trajectory. In order to do so, we're going to neglect the curvature of the road and assume it is locally straight. Regarding longitudinal acceleration, we make the additional assumption that our heading is pretty much aligned with the road. This allows us to say that S doubled dots is the longitude acceleration of the car. Therefore, we need to check that at any point of the trajectory, this acceleration is less than the maximum acceleration that the engine would need to supply, and more than the maximum braking deceleration of the car. Right now, this could be a fixed value. In real life however, this should probably be computed using information about the friction of the road. Similarly, for lateral acceleration, we can check that all d double dot values are less than a fixed lateral acceleration value that can be set for comfort, or to avoid any risk of rollover in our car. Regarding steering angle, the bicycle model tells us that there's a nice relationship between the steering angle of the car and the radius of the circle of curvature, where L is the distance between the wheel axis and R is the circle radius. The curvature is then given by this formula, and therefore the maximum curvature allowed at any point of the trajectory is given by this equation. If you remember, in the reading assignment about [inaudible] the curvature for path is defined like this, where Î”Ï†i is the heading difference between two points of the trajectory and Î”Ï‡i is the distance between them. Finally the velocity is checked against values specified by the map or the behavioral layer. For example, we could use the speed limit of the road in most cases. That gives us a maximal velocity but also sometimes, we need a minimal velocity like on highways where we don't want to drive too slow or we're backing up, negative S dot is not allowed.
<img src="./outputs/tg74.png" width="500" height="500" />


## Putting it All Together

In most cases the behavioral layer might not send an exact end configuration to reach but rather an approximate one like that. Therefore, we want to identify a varied goal state that leads our vehicle inthe approximate end config that's being sent to us here in the other lane.Now, Thats where our sampling based approach comes in handy.We don't know for sure what a good end state is.
<img src="./outputs/tg75.png" width="500" height="500" />

Even if we recive an exact S and D co-ordinate from the behavior layer,we still have to find a good end velocity and acceleration for our car.Most times S and D aren't fixed either.Therefore, we want to sample a large number of end configs are on the approximate desired position that we wish to drive our car to.
<img src="./outputs/tg76.png" width="500" height="500" />

And we generate the corresponsding Jerk minimizing trajectories for each goal config.The , we discard all non-drivable trajectories,all grajectories that collide with the road boundaries, or with other vehicles or pedestrians as predicted by the prediction layer.
<img src="./outputs/tg77.png" width="500" height="500" />

Now we have a nice set of drivable trajectories which are all collision free and  Jerk optimal in each dimension with respect to the start and goal configs. We need to decide which one the car should follow.That means we need to rank them,which we are going to do by defining a cost function.

What makes sense as a cost function?
* First consider Jerk: For each pair of end configurations,a quintic is jerk optimal. But given that all the trajectories obtained are different final config,may be somne are better than others.In addition to that,we want to consider the longitudinal versus lateral Jerk.

What is worse for comfort? Turnsout that side to side Jerk is more comfortable.So we want to prioritize minimizing that over longitudinal Jerk.

<img src="./outputs/tg78.png" width="300" height="300" />

* Next, consider distance to the obstacles: Given these 2 configs,we would prefer being farther away from the other vehicle to a situation where we are right next to their rear bumber
<img src="./outputs/tg79.png" width="300" height="300" />

* Also we want to consider distance to center line.We might prefer to be close to the center of the lane.So, in choosing between these two scenarios,
<img src="./outputs/tg80.png" width="300" height="300" />
 may be we would prefer the one on the right
 
* Finally it's time to go.All else being equal, given the final s, we would prefer to arrive at our destination earlier rather than later.
<img src="./outputs/tg81.png" width="300" height="300" />

And there are plenty of other cost functions one might think of.The tricky part is balancing all of these costs correctly.Often our goals conflict, So trajectory that minimizes lateral Jerk may not be the one that keeps us closest to the center line.In practice,most of the hard work is in the details of balancing these cost functions.
Now that we have defined all these cost functions and combined them into one weighted cost functions, we can look back at the scenario we just saw and use the supplied cost functions to calculate the cost for each trajectory.
Remember to consider width and length of vehicle when considering distance to obstacle. for eaxmples this trajectory place vehciles close to each other

<img src="./outputs/tg82.png" width="500" height="500" />


In the image below trajectories marked red and orange have high cost because either they place the vehicle close to lane boundary or to another vehicle.
The trajectories in yello are medium cost as they either introduce too much jerk or not as far along (faster to goal). Trajectory in blue end up getting selected for this scenario
<img src="./outputs/tg83.png" width="500" height="500" />



## Polynomial Trajectory Generation
Here is a link to a paper (below) titled "Optimal Trajectory Generation for Dynamic Street Scenarios in a Frenet Frame". It is short and discusses some interesting (and potentially useful) topics like:

* Cost Functions.
* Differences between high speed and low speed trajectory generation.
* Implementation of specific maneuvers relevant to highway driving like following, merging, and velocity keeping.
* How to combining lateral and longitudinal trajectories.
* A derivation of the transformation from Frenet coordinates to global coordinates (in the appendix).


Supporting Materials:[ Optimal Trajectory Generation For Dynamic Street Scenarios In A Frenet Frame](./werling-optimal-trajectory-generation-for-dynamic-street-scenarios-in-a-frenet-frame.pdf)

## Polynomial Playground (making PTG work)
Before you begin the final project let's play around with cost function design and weight tweaking. In the Python code you will be provided, you will have a working Polynomial Trajectory Generator. You can see it in action by doing the following:

### Getting Started
1. Download the project code by clicking [TrajectoryExercise2](./trajectoryexercise2-python3.zip).
2. From the project's directory, run python evaluate_ptg.py. You should see a plot similar to the one below. This plot shows the s (x-axis) and d(y-axis) trajectories followed by a vehicle in traffic (red) and a self driving car (blue).

<img src="./outputs/tg84.png" width="500" height="500" />

In this situation, the self driving car was trying to get behind the target vehicle, but the cost functions it was using weren't weighted appropriately and so it didn't behave as expected.

### Fixing the Problem(s)
There are 5 files in the provided code. You'll probably want to start by modifying cost function weights in ptg.py but may also want to add cost functions of your own. As a bonus challenge try implementing this same code in C++.

File Descriptions

1. ptg.py - The primary code for generating a polynomial trajectory for some constraints. This is also where weights are assigned to cost functions. Adjusting these weights (and possibly adding new cost functions), can have a big effect on vehicle behavior.
2. cost_functions.py - This file contains many cost functions which are used in ptg.py when selecting the best trajectory. Some cost functions aren't yet implemented...
3. evaluate_ptg.py - This file sets a start state, goal, and traffic conditions and runs the PTG code. Feel free to modify the goal, add traffic, etc... to test your vehicle's trajectory generation ability.
4. constants.py - constants like speed limit, vehicle size, etc...
5. helpers.py - helper functions used by other files.

Conclusion 

In this lesson, we saw several approaches to the trajectory generation problem. None of them is the best for every situation that a self-driving car will encounter. In reality, most self-driving cars have several trajectory planners they can use depending on the situation. A car may use hybrid A\*  in parking lots, polynomial trajectory generation on low traffic highways, and maybe several others for situations like intersections, high traffic, et cetera. In the final project, we wii choose to implement trajectory generation. Most probably polynomial trajectory generation is well-suited for this problem. 